In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [78]:
# Node class
class Node:
  def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
    '''constructor for the node of the decision tree'''

    # for decision node
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.info_gain = info_gain

    # for leaf node
    self.value = value

In [89]:
# tree class
class DecisionTreeClassifier:
    def __init__(self, min_sample_split=3, max_depth=2):
        '''Constructor of the decision tree class'''
        self.root = None
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        '''Recursive function to build the tree'''
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        # Stop splitting if conditions are met
        if num_samples >= self.min_sample_split and curr_depth < self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split and best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])

        # Compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        '''Finds the best split'''
        best_split = {"info_gain": -float("inf")}

        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)

                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")

                    if curr_info_gain > best_split["info_gain"]:
                        best_split = {
                            "feature_index": feature_index,
                            "threshold": threshold,
                            "dataset_left": dataset_left,
                            "dataset_right": dataset_right,
                            "info_gain": curr_info_gain
                        }

        return best_split if best_split["info_gain"] > 0 else None

    def split(self, dataset, feature_index, threshold):
        '''Splits dataset based on a feature index and threshold'''
        dataset_left = dataset[dataset[:, feature_index] <= threshold]
        dataset_right = dataset[dataset[:, feature_index] > threshold]

        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, method='gini'):
        '''Computes information gain using gini or entropy'''
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)

        if method == 'gini':
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))

        return gain

    def gini_index(self, y):
        '''Computes Gini index'''
        class_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def entropy(self, y):
        '''Computes entropy'''
        class_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-9))

    def calculate_leaf_value(self, Y):
        '''Determines leaf value by majority class'''
        return np.bincount(Y.astype(int)).argmax()

    def fit(self, X, Y):
        '''Fits the decision tree to the data'''
        dataset = np.column_stack((X, Y))
        self.root = self.build_tree(dataset)

    def predict(self, X):
        '''Predicts class labels for input samples'''
        return np.array([self.traverse_tree(x, self.root) for x in X])

    def traverse_tree(self, x, node):
        '''Traverses the tree for a single prediction'''
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)

In [90]:
# fetching data
df = pd.read_csv("/content/drive/MyDrive/ML/Dataset/Iris.csv")

df["Species"] = df["Species"].map({'Iris-setosa': 1, 'Iris-versicolor': 2, 'Iris-virginica': 3})
# df = df.astype(float)

df_1 = df["Species"]

df.head()
# print(df["Species"].unique())
# print(df.isnull().sum())
# (df == " ").sum()
# print(df['Species'].value_counts())

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,1
1,2,4.9,3.0,1.4,0.2,1
2,3,4.7,3.2,1.3,0.2,1
3,4,4.6,3.1,1.5,0.2,1
4,5,5.0,3.6,1.4,0.2,1


In [91]:
# separating data
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 23)

In [92]:
# making tree
classifier = DecisionTreeClassifier(min_sample_split=2, max_depth=5)
classifier.fit(X_train, Y_train)

In [93]:
# predicting data
pred = classifier.predict(X_test).reshape(-1, 1)
print(np.mean(pred == Y_test))

1.0


In [84]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [85]:
# Create and train Decision Tree model
classifier = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_split=2, random_state=42)
classifier.fit(X_train, Y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print(f"Decision Tree Accuracy: {accuracy:.4f}")

Decision Tree Accuracy: 1.0000
